In [1]:
###NEEDS GIT DOCUMENTATION
path='G:/Shared drives/BeStarsMiMeS/ScienceFiles/' #Patrick's google file stream path
#import LSDpy.lsdpy
#import specpolFlow as pol


import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import itertools

import pyRaven as rav
import scipy

In [2]:
#Accessing the google sheets

sheet_id = '1M6y1Wnsrc-w5FjUMfKaSFa_-foIDAaMe8W4lYNWnWyk'
sheet_name = 'Stars'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
StarData=pd.read_csv(url)

sheet_name = 'Observations'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
Observations=pd.read_csv(url)

In [3]:
StarData

,HD,Name,Nobs,T,log(g),ModelCode,Adopted-vsini,Halpha shape,Notes,vsini-estimate,SB-flag,SB9 catalog,pyRavenFlag,Asif Hybrid Over/Underfill,i_min [deg],Other Notes,JonNotes,Unnamed: 17
0,NaN,hd6226,1,15000,4.0,T15000g40,120.00,absorption,"no disk during observation, rotational modulat...",90.0,NaN,NaN,1.0,under,10.369760,NaN,Definitely classical Be. Line profile weirdnes...,NaN
1,7636.0,hd7636,2,15000,4.0,T15000g40,220.00,asymetric single,binary? no tess observation,150.0,NaN,NaN,2.0,under,17.457603,NaN,Definitely classical Be. LPVs probably from pu...,NaN
2,10144.0,hd10144,1,20000,3.5,T20000g35,245.31,bowl,binary or pulsator,245.0,NaN,NaN,1.0,over,29.340582,NaN,Achernar! Brightest Be star in the sky. Nice a...,NaN
3,10516.0,hd10516,1,25000,4.0,T25000g40,382.04,asymetric double,sdO companion https://iopscience.iop.org/artic...,280.0,1.0,SB2,1.0,NaN,34.055798,NaN,"phi Per, classical Be. The strong asymmetry in...",NaN
4,11415.0,hd11415,1,18000,4.0,T18000g40,48.00,absorption,not a binary https://iopscience.iop.org/articl...,60.0,NaN,NaN,NaN,under,6.892103,NaN,"eps Cas. Probably not be a Be star, No sign of...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,217891.0,hd217891,5,15000,3.0,T15000g30,95.00,single,not a binary https://iopscience.iop.org/articl...,120.0,NaN,NaN,1.0,NaN,13.886540,NaN,bet Psc. consistently has strong emission. Qui...,NaN
74,221507.0,hd221507,1,10500,4.0,T10500g40,40.00,absorption,Not a Be star,NaN,NaN,NaN,NaN,under,4.588566,NaN,"yeah, not a Be star. What's interesting though...",NaN
75,224686.0,hd224686,2,13000,4.0,T13000g40,286.16,shell,NaN,290.0,NaN,NaN,1.0,under,35.450543,NaN,typical late-type Be star. Shouldn't have to w...,NaN
76,239712.0,hd239712,1,21000,4.0,T21000g40,304.86,asymetric double,NaN,290.0,NaN,NaN,1.0,NaN,35.450543,NaN,distortion in He 6678 probably due to asymmetr...,NaN


In [4]:
genparam = {
    'lambda0':5811.969,    # the central wavelength of the transition
    'vsini':200,         # the projected rotational velocity
    'vdop':10.0,          # the thermal broadening
    'av':0.05,             # the damping coefficient of the Voigt profile
    'bnu':1.5,             # the slope of the source function with respect to vertical optical depth
    'logkappa':np.log(1.3),          # the line strength parameter
    'Bpole':1.0e1,         # the dipolar field strength
    'incl':np.pi/4,      # the inclination of the rotational axis to the line of sight
    'beta':np.pi/4,      # the obliquity of the magnetic axis to the rotational axis
    'phase':0.0,     # the rotational phase
    'ndop':int(100),       # the number of sample point per doppler width for the wavelength array
    }
unnoparam = {
    'down':[0.5,0, 0.5],   # the s, j, l of the lower level
    'up':[0.5, 1, 0.5]    # the s, j, l of the upper level
    }
weakparam = {
    'geff':1.0,
    }
param={'general' : genparam,
    'unno' : unnoparam,
    'weak' : weakparam}
param['general'].update({'res': 65000.})
param['general'].update({'vmac': 10.})

## NEW

In [6]:
%%capture

models=['hybrid_maskdepth0.01_tellClean','hybrid_CleanAsif+tellClean','hybrid_CleanAsifTweak+tellClean','normalized_CleanAsif+tellClean',
 'normalized_CleanAsifTweak+tellClean']


for j in range(len(models)):
    with PdfPages('{}05-RavenSetup/{}_fit.pdf'.format(path,models[j])) as pdf:
        for i,StarInfo in StarData.iterrows():
            if StarInfo['pyRavenFlag']==1 or StarInfo['pyRavenFlag']==2:
                star=StarInfo['Name']
                fname='{}05-RavenSetup/{}/{}.h5'.format(path,models[j],star)
                datapacket=rav.data.read_packet(fname)
                vsini=StarInfo['vsini-estimate']
                
                nobs=datapacket.nobs
                param_to_fit = [['logkappa','vmac','vrad']]
                guess = [[np.log(0.51),30,[0]*nobs]]
                bounds = [[(None,None),(0,40),[(-300,300)]*nobs]]

                genparam = {
                    'lambda0':5811.969,    # the central wavelength of the transition
                    'vsini':vsini,         # the projected rotational velocity
                    'vdop':10.0,          # the thermal broadening
                    'av':0.05,             # the damping coefficient of the Voigt profile
                    'bnu':1.5,             # the slope of the source function with respect to vertical optical depth
                    'logkappa':np.log(1.3),          # the line strength parameter
                    'Bpole':1.0e1,         # the dipolar field strength
                    'incl':np.pi/4,      # the inclination of the rotational axis to the line of sight
                    'beta':np.pi/4,      # the obliquity of the magnetic axis to the rotational axis
                    'phase':0.0,     # the rotational phase
                    'ndop':int(100),       # the number of sample point per doppler width for the wavelength array
                    }
                unnoparam = {
                    'down':[0.5,0, 0.5],   # the s, j, l of the lower level
                    'up':[0.5, 1, 0.5]    # the s, j, l of the upper level
                    }
                weakparam = {
                    'geff':1.0,
                    }
                param={'general' : genparam,
                    'unno' : unnoparam,
                    'weak' : weakparam}
                param['general'].update({'res': 65000.})
                param['general'].update({'vmac': 10.})
                param['general'].update({'vrad': [-40]*nobs})

                parameters=[param]

                xs=[]
                ys=[]
                ys_err=[]
                for o in range(datapacket.nobs):
                    xs.append(datapacket.scaled.lsds[o].vel)
                    ys.append(datapacket.scaled.lsds[o].specI)
                    ys_err.append(datapacket.scaled.lsds[o].specSigI)

                lsds=[xs,ys,ys_err]

                res,star2=rav.fitparams.fitting(param_to_fit,parameters,lsds,guess,bounds)
                
                fit1=rav.fitparams.param_to_model(star2,lsds)

                fig, ax = plt.subplots(1,1,sharex=True,figsize=(10,5))

                ax.plot(xs[0],fit1[0],color='black',linestyle='dashed')
                ax.plot(xs[0],ys[0],linewidth=8,color='black',alpha=0.3,zorder=0)

                rav.params.parameters(star2[0]).write('{}05-RavenSetup/params/{}/{}.json'.format(path,models[j],star))
                
                pdf.savefig(fig)
                print(models[j],star)

## OLD

In [5]:
%%capture
with PdfPages('{}05-RavenSetup/hybrid0.02FitModel.pdf'.format(path)) as pdf:
    for i,StarInfo in StarData.iterrows():
        if StarInfo['pyRavenFlag']==1 or StarInfo['pyRavenFlag']==2:
            star=StarInfo['Name']
            fname='{}05-RavenSetup/hybrid_maskdepth0.02_tellClean/{}.h5'.format(path,star)
            datapacket=datapacket=rav.data.read_packet(fname)
            guess=np.array([1.3,datapacket.vsini,30])

            genparam = {
                'lambda0':5811.969,    # the central wavelength of the transition
                'vsini':200,         # the projected rotational velocity
                'vdop':10.0,          # the thermal broadening
                'av':0.05,             # the damping coefficient of the Voigt profile
                'bnu':1.5,             # the slope of the source function with respect to vertical optical depth
                'logkappa':np.log(1.3),          # the line strength parameter
                'Bpole':1.0e1,         # the dipolar field strength
                'incl':np.pi/4,      # the inclination of the rotational axis to the line of sight
                'beta':np.pi/4,      # the obliquity of the magnetic axis to the rotational axis
                'phase':0.0,     # the rotational phase
                'ndop':int(100),       # the number of sample point per doppler width for the wavelength array
                }
            unnoparam = {
                'down':[0.5,0, 0.5],   # the s, j, l of the lower level
                'up':[0.5, 1, 0.5]    # the s, j, l of the upper level
                }
            weakparam = {
                'geff':1.0,
                }
            param={'general' : genparam,
                'unno' : unnoparam,
                'weak' : weakparam}
            param['general'].update({'res': 65000.})
            param['general'].update({'vmac': 10.})

            xs=[]
            ys=[]
            for n in range(datapacket.nobs):
                guess=np.append(guess, 10)
                xs.append(datapacket.scaled.lsds[n].vel)
                ys.append(datapacket.scaled.lsds[n].specI)

            parameters,covariance,modelout = rav.fitparams.fitdata(xs,ys,guess,param)

            fig, ax = plt.subplots(1,1, figsize=(5,5))
            for n in range(datapacket.nobs):
                x1=datapacket.scaled.lsds[n].vel#+datapacket.vrad[n]
                y1=datapacket.scaled.lsds[n].specI
                if n==0:
                    ax.plot(x1,y1,c='C0',label='Data')
                else:
                    ax.plot(x1,y1,c='C0')
            x2=datapacket.scaled.lsds[0].vel#+datapacket.vrad[0]
            ax.plot(x2,modelout[0][0],c='C1',label='Fit Model')
            ax.legend()
            ax.set_title(star)

            #row=np.append(star,parameters)
            #pd.DataFrame(row).T.to_csv('{}05-RavenSetup/fitparameters_hybrid.csv'.format(path),mode='a',index=False,header=False)

            #param['general'].update({'logkappa': np.log(parameters[0])})
            #param['general'].update({'vsini': parameters[1]})
            #param['general'].update({'vmac': parameters[2]})

            #param=rav.params.parameters(param)
            #param.write('{}05-RavenSetup/params_hybrid/{}.json'.format(path,star))

            pdf.savefig(fig)

In [12]:
%%capture
with PdfPages('{}05-RavenSetup/normSuperCleanFitModel.pdf'.format(path)) as pdf:
    for i,StarInfo in StarData.iterrows():
        if StarInfo['pyRavenFlag']==2:
            star=StarInfo['Name']
            fname='{}05-RavenSetup/normalized_SuperCleanAsif_tellClean/{}.h5'.format(path,star)
            datapacket=datapacket=rav.data.read_packet(fname)
            guess=np.array([1.3,datapacket.vsini,30])

            genparam = {
                'lambda0':5811.969,    # the central wavelength of the transition
                'vsini':200,         # the projected rotational velocity
                'vdop':10.0,          # the thermal broadening
                'av':0.05,             # the damping coefficient of the Voigt profile
                'bnu':1.5,             # the slope of the source function with respect to vertical optical depth
                'logkappa':np.log(1.3),          # the line strength parameter
                'Bpole':1.0e1,         # the dipolar field strength
                'incl':np.pi/4,      # the inclination of the rotational axis to the line of sight
                'beta':np.pi/4,      # the obliquity of the magnetic axis to the rotational axis
                'phase':0.0,     # the rotational phase
                'ndop':int(100),       # the number of sample point per doppler width for the wavelength array
                }
            unnoparam = {
                'down':[0.5,0, 0.5],   # the s, j, l of the lower level
                'up':[0.5, 1, 0.5]    # the s, j, l of the upper level
                }
            weakparam = {
                'geff':1.0,
                }
            param={'general' : genparam,
                'unno' : unnoparam,
                'weak' : weakparam}
            param['general'].update({'res': 65000.})
            param['general'].update({'vmac': 10.})

            xs=[]
            ys=[]
            for n in range(datapacket.nobs):
                guess=np.append(guess, 10)
                xs.append(datapacket.scaled.lsds[n].vel)
                ys.append(datapacket.scaled.lsds[n].specI)

            parameters,covariance,modelout = rav.fitparams.fitdata(xs,ys,guess,param)

            fig, ax = plt.subplots(1,1, figsize=(5,5))
            for n in range(datapacket.nobs):
                x1=datapacket.scaled.lsds[n].vel#+datapacket.vrad[n]
                y1=datapacket.scaled.lsds[n].specI
                if n==0:
                    ax.plot(x1,y1,c='C0',label='Data')
                else:
                    ax.plot(x1,y1,c='C0')
            x2=datapacket.scaled.lsds[0].vel#+datapacket.vrad[0]
            ax.plot(x2,modelout[0][0],c='C1',label='Fit Model')
            ax.legend()
            ax.set_title(star)

            row=np.append(star,parameters)
            pd.DataFrame(row).T.to_csv('{}05-RavenSetup/fitparameters_norm.csv'.format(path),mode='a',index=False,header=False)

            param['general'].update({'logkappa': np.log(parameters[0])})
            param['general'].update({'vsini': parameters[1]})
            param['general'].update({'vmac': parameters[2]})

            param=rav.params.parameters(param)
            param.write('{}05-RavenSetup/params_norm/{}.json'.format(path,star))

            pdf.savefig(fig)

In [ ]:
%%capture
with PdfPages('{}hybrid.02FitModel_fixedvsini.pdf'.format(path)) as pdf:
    for i,StarInfo in StarData.iterrows():
        if StarInfo['pyRavenFlag']==2.0:
            star=StarInfo['Name']
            fname='{}05-RavenSetup/hybrid_maskdepth0.02_tellClean/{}.h5'.format(path,star)
            datapacket=datapacket=rav.data.read_packet(fname)
            guess=np.array([1.3,30])

            genparam = {
                'lambda0':5811.969,    # the central wavelength of the transition
                'vsini':200,         # the projected rotational velocity
                'vdop':10.0,          # the thermal broadening
                'av':0.05,             # the damping coefficient of the Voigt profile
                'bnu':1.5,             # the slope of the source function with respect to vertical optical depth
                'logkappa':np.log(1.3),          # the line strength parameter
                'Bpole':1.0e1,         # the dipolar field strength
                'incl':np.pi/4,      # the inclination of the rotational axis to the line of sight
                'beta':np.pi/4,      # the obliquity of the magnetic axis to the rotational axis
                'phase':0.0,     # the rotational phase
                'ndop':int(100),       # the number of sample point per doppler width for the wavelength array
                }
            unnoparam = {
                'down':[0.5,0, 0.5],   # the s, j, l of the lower level
                'up':[0.5, 1, 0.5]    # the s, j, l of the upper level
                }
            weakparam = {
                'geff':1.0,
                }
            param={'general' : genparam,
                'unno' : unnoparam,
                'weak' : weakparam}
            param['general'].update({'res': 65000.})
            param['general'].update({'vmac': 10.})

            xs=[]
            ys=[]
            for n in range(datapacket.nobs):
                guess=np.append(guess, 10)
                xs.append(datapacket.scaled.lsds[n].vel)
                ys.append(datapacket.scaled.lsds[n].specI)

            parameters,covariance,modelout = rav.fitparams.fitdata_novsini(xs,ys,guess,param)

            fig, ax = plt.subplots(1,1, figsize=(5,5))
            for n in range(datapacket.nobs):
                x1=datapacket.scaled.lsds[n].vel#+datapacket.vrad[n]
                y1=datapacket.scaled.lsds[n].specI
                if n==0:
                    ax.plot(x1,y1,c='C0',label='Data')
                else:
                    ax.plot(x1,y1,c='C0')
            x2=datapacket.scaled.lsds[0].vel#+datapacket.vrad[0]
            ax.plot(x2,modelout[0][0],c='C1',label='Fit Model')
            ax.legend()
            ax.set_title(star)

            parameters=np.append(parameters,datapacket.vsini)
            row=np.append(star,parameters)
            pd.DataFrame(row).T.to_csv('{}05-RavenSetup/fitparameters_fixedvsini.csv'.format(path),mode='a',index=False,header=False)

            #param['general'].update({'logkappa': np.log(parameters[0])})
            #param['general'].update({'vsini': parameters[1]})
            #param['general'].update({'vmac': parameters[2]})

            #param=rav.params.parameters(param)
            #param.write('{}05-RavenSetup/params/{}.json'.format(path,star))

            pdf.savefig(fig)

In [ ]:
%%capture
with PdfPages('{}normSuperCleanFitModel_fixedvsini.pdf'.format(path)) as pdf:
    for i,StarInfo in StarData.iterrows():
        if StarInfo['pyRavenFlag']==2.0 or StarInfo['pyRavenFlag']==1.0:
            star=StarInfo['Name']
            fname='{}05-RavenSetup/normalized_SuperCleanAsif_tellClean/{}.h5'.format(path,star)
            datapacket=datapacket=rav.data.read_packet(fname)
            guess=np.array([1.3,30])
            genparam = {
                'lambda0':5811.969,    # the central wavelength of the transition
                'vsini':200,         # the projected rotational velocity
                'vdop':10.0,          # the thermal broadening
                'av':0.05,             # the damping coefficient of the Voigt profile
                'bnu':1.5,             # the slope of the source function with respect to vertical optical depth
                'logkappa':np.log(1.3),          # the line strength parameter
                'Bpole':1.0e1,         # the dipolar field strength
                'incl':np.pi/4,      # the inclination of the rotational axis to the line of sight
                'beta':np.pi/4,      # the obliquity of the magnetic axis to the rotational axis
                'phase':0.0,     # the rotational phase
                'ndop':int(100),       # the number of sample point per doppler width for the wavelength array
                }
            unnoparam = {
                'down':[0.5,0, 0.5],   # the s, j, l of the lower level
                'up':[0.5, 1, 0.5]    # the s, j, l of the upper level
                }
            weakparam = {
                'geff':1.0,
                }
            param={'general' : genparam,
                'unno' : unnoparam,
                'weak' : weakparam}
            param['general'].update({'res': 65000.})
            param['general'].update({'vmac': 10.})

            xs=[]
            ys=[]
            for n in range(datapacket.nobs):
                guess=np.append(guess, 10)
                xs.append(datapacket.scaled.lsds[n].vel)
                ys.append(datapacket.scaled.lsds[n].specI)

            parameters,covariance,modelout = rav.fitparams.fitdata_novsini(xs,ys,guess,param)
            fig, ax = plt.subplots(1,1, figsize=(5,5))
            for n in range(datapacket.nobs):
                x1=datapacket.scaled.lsds[n].vel#+datapacket.vrad[n]
                y1=datapacket.scaled.lsds[n].specI
                if n==0:
                    ax.plot(x1,y1,c='C0',label='Data')
                else:
                    ax.plot(x1,y1,c='C0')
            x2=datapacket.scaled.lsds[0].vel#+datapacket.vrad[0]
            ax.plot(x2,modelout[0][0],c='C1',label='Fit Model')
            ax.legend()
            ax.set_title(star)

            #parameters=np.append(parameters,datapacket.vsini)
            #row=np.append(star,parameters)
            #pd.DataFrame(row).T.to_csv('{}05-RavenSetup/fitparameters_norm_fixedvsini.csv'.format(path),mode='a',index=False,header=False)

            #param['general'].update({'logkappa': np.log(parameters[0])})
            #param['general'].update({'vsini': parameters[1]})
            #param['general'].update({'vmac': parameters[2]})

            #param=rav.params.parameters(param)
            #param.write('{}05-RavenSetup/params/{}.json'.format(path,star))

            pdf.savefig(fig)

In [13]:
%%capture
with PdfPages('{}05-RavenSetup/normSuperCleanTweakFitModel.pdf'.format(path)) as pdf:
    for i,StarInfo in StarData.iterrows():
        if StarInfo['pyRavenFlag']==2:
            star=StarInfo['Name']
            fname='{}05-RavenSetup/normalized_SuperCleanAsifTweak_tellClean/{}.h5'.format(path,star)
            datapacket=datapacket=rav.data.read_packet(fname)
            guess=np.array([1.3,datapacket.vsini,30])

            genparam = {
                'lambda0':5811.969,    # the central wavelength of the transition
                'vsini':200,         # the projected rotational velocity
                'vdop':10.0,          # the thermal broadening
                'av':0.05,             # the damping coefficient of the Voigt profile
                'bnu':1.5,             # the slope of the source function with respect to vertical optical depth
                'logkappa':np.log(1.3),          # the line strength parameter
                'Bpole':1.0e1,         # the dipolar field strength
                'incl':np.pi/4,      # the inclination of the rotational axis to the line of sight
                'beta':np.pi/4,      # the obliquity of the magnetic axis to the rotational axis
                'phase':0.0,     # the rotational phase
                'ndop':int(100),       # the number of sample point per doppler width for the wavelength array
                }
            unnoparam = {
                'down':[0.5,0, 0.5],   # the s, j, l of the lower level
                'up':[0.5, 1, 0.5]    # the s, j, l of the upper level
                }
            weakparam = {
                'geff':1.0,
                }
            param={'general' : genparam,
                'unno' : unnoparam,
                'weak' : weakparam}
            param['general'].update({'res': 65000.})
            param['general'].update({'vmac': 10.})

            xs=[]
            ys=[]
            for n in range(datapacket.nobs):
                guess=np.append(guess, 10)
                xs.append(datapacket.scaled.lsds[n].vel)
                ys.append(datapacket.scaled.lsds[n].specI)

            parameters,covariance,modelout = rav.fitparams.fitdata(xs,ys,guess,param)

            fig, ax = plt.subplots(1,1, figsize=(5,5))
            for n in range(datapacket.nobs):
                x1=datapacket.scaled.lsds[n].vel#+datapacket.vrad[n]
                y1=datapacket.scaled.lsds[n].specI
                if n==0:
                    ax.plot(x1,y1,c='C0',label='Data')
                else:
                    ax.plot(x1,y1,c='C0')
            x2=datapacket.scaled.lsds[0].vel#+datapacket.vrad[0]
            ax.plot(x2,modelout[0][0],c='C1',label='Fit Model')
            ax.legend()
            ax.set_title(star)

            row=np.append(star,parameters)
            pd.DataFrame(row).T.to_csv('{}05-RavenSetup/fitparameters_normTweak.csv'.format(path),mode='a',index=False,header=False)

            param['general'].update({'logkappa': np.log(parameters[0])})
            param['general'].update({'vsini': parameters[1]})
            param['general'].update({'vmac': parameters[2]})

            param=rav.params.parameters(param)
            param.write('{}05-RavenSetup/params_normTweak/{}.json'.format(path,star))

            pdf.savefig(fig)

KeyboardInterrupt: 

## Testing

In [ ]:
def fitdata_novsini(param,DataPacket,guess):
  '''
  This function fits a set of LSD profiles using scipy's curve fit function.

  Inputs:
    param - input parameter dictionary
    DataPacket - input DataPacket with real data
    guess - array of guess values for kappa and vmac. Ex: np.array([1.3,30])

   Outputs:
    parameters - array of fit parameters
    covariance - covariance matrix of the fit
    modelout - the best fit model

  '''
  def model(v,kappa,vmac):
      '''
      This function creates the line profile model that will be fit to the observed profile

      Inputs:
        kappa - value of kappa that the walker is on in parameter space
        vmac - value of vmac that the walker is on in parameter space
        v - velocity array of the actual data

      Outputs:
        f - line profile model using the weak field method at the walker's position in parameter space
      '''
      param['general']['vmac']=vmac
      param['general']['logkappa']=np.log(kappa)

      #pyRaven's weakfield disk integration function
      model=rav.diskint2.analytical(param,False)

      #interpolating the model to be size MCMC wants
      f=np.interp(v,model['vel'],model['flux'])
      return(f)
    
  param['general']['vsini']=DataPacket.vsini

  x=DataPacket.original.lsds[0].vel-DataPacket.vrad[0]
  y=DataPacket.original.lsds[0].specI
  if DataPacket.nobs!=1:
    for i in range(1,DataPacket.nobs):
      x=np.append(x,DataPacket.original.lsds[i].vel-DataPacket.vrad[i])
      y=np.append(y,DataPacket.original.lsds[i].specI)

  parameters,covariance = scipy.optimize.curve_fit(model,x,y,guess)

  modelout=model(x,parameters[0],parameters[1])
  modelout=modelout[:DataPacket.original.lsds[0].vel.size]
  return parameters,covariance,modelout


for i,StarInfo in StarData.iterrows():
    if StarInfo['pyRavenFlag']==StarInfo['AsifgoodFlag']==1.0:
        star=StarInfo['Name']
        fname='{}05-RavenSetup/hybrid_maskdepth0.02_tellClean/{}.h5'.format(path,star)
        datapacket=datapacket=rav.data.read_packet(fname)
        guess=np.array([1.3,30])

        parameters,covariance,modelout = fitdata_novsini(param,datapacket,guess)

        fig, ax = plt.subplots(1,1, figsize=(5,5))
        for n in range(datapacket.nobs):
            x1=datapacket.original.lsds[n].vel+datapacket.vrad[n]
            y1=datapacket.original.lsds[n].specI
            if n==0:
                ax.plot(x1,y1,c='C0',label='Data')
            else:
                ax.plot(x1,y1,c='C0')
        x2=datapacket.original.lsds[0].vel+datapacket.vrad[0]
        ax.plot(x1,modelout,c='C1',label='Fit Model')
        ax.legend()
        ax.set_title(star)

        parameters=np.append(parameters,datapacket.vsini)
        row=np.append(star,parameters)
        print(parameters)
        if i==2:
            break




In [ ]:
def fitdata_novsini(param,DataPacket,guess):
  '''
  This function fits a set of LSD profiles using scipy's curve fit function.

  Inputs:
    param - input parameter dictionary
    DataPacket - input DataPacket with real data
    guess - array of guess values for kappa and vmac. Ex: np.array([1.3,30])

   Outputs:
    parameters - array of fit parameters
    covariance - covariance matrix of the fit
    modelout - the best fit model

  '''
  def model(v,kappa,vmac):
      '''
      This function creates the line profile model that will be fit to the observed profile

      Inputs:
        kappa - value of kappa that the walker is on in parameter space
        vmac - value of vmac that the walker is on in parameter space
        v - velocity array of the actual data

      Outputs:
        f - line profile model using the weak field method at the walker's position in parameter space
      '''
      param['general']['vmac']=vmac
      param['general']['logkappa']=np.log(kappa)

      #pyRaven's weakfield disk integration function
      model=rav.diskint2.analytical(param,False)

      #interpolating the model to be size MCMC wants
      f=np.interp(v,model['vel'],model['flux'])
      return(f)
    
  param['general']['vsini']=DataPacket.vsini

  x=DataPacket.scaled.lsds[0].vel#+DataPacket.vrad[0]
  y=DataPacket.scaled.lsds[0].specI
  if DataPacket.nobs!=1:
    for i in range(1,DataPacket.nobs):
      x=np.append(x,DataPacket.scaled.lsds[i].vel)#+DataPacket.vrad[i])
      y=np.append(y,DataPacket.scaled.lsds[i].specI)

  parameters,covariance = scipy.optimize.curve_fit(model,x,y,guess)

  modelout=model(x,parameters[0],parameters[1])
  modelout=modelout[:DataPacket.scaled.lsds[0].vel.size]
  return parameters,covariance,modelout


for i,StarInfo in StarData.iterrows():
    if StarInfo['pyRavenFlag']==StarInfo['AsifgoodFlag']==1.0:
        star=StarInfo['Name']
        fname='{}05-RavenSetup/hybrid_maskdepth0.02_tellClean/{}.h5'.format(path,star)
        datapacket=datapacket=rav.data.read_packet(fname)
        guess=np.array([1.3,30])

        parameters,covariance,modelout = fitdata_novsini(param,datapacket,guess)

        fig, ax = plt.subplots(1,1, figsize=(5,5))
        for n in range(datapacket.nobs):
            x1=datapacket.scaled.lsds[n].vel#+datapacket.vrad[n]
            y1=datapacket.scaled.lsds[n].specI
            if n==0:
                ax.plot(x1,y1,c='C0',label='Data')
            else:
                ax.plot(x1,y1,c='C0')
        x2=datapacket.scaled.lsds[0].vel#+datapacket.vrad[0]
        ax.plot(x2,modelout,c='C1',label='Fit Model')
        ax.legend()
        ax.set_title(star)

        parameters=np.append(parameters,datapacket.vsini)
        row=np.append(star,parameters)
        print(parameters)
        if i==3:
            break




In [ ]:
fit=pd.read_csv('{}14-RavenFittingComparisons/fitparameters.csv'.format(path),header=None)
fitfixed=pd.read_csv('{}14-RavenFittingComparisons/fitparameters_fixedvsini.csv'.format(path),header=None)

def model1(v,kappa,vmac):
    '''
    This function creates the line profile model that will be fit to the observed profile

    Inputs:
    kappa - value of kappa that the walker is on in parameter space
    vmac - value of vmac that the walker is on in parameter space
    v - velocity array of the actual data

    Outputs:
    f - line profile model using the weak field method at the walker's position in parameter space
    '''
    param['general']['vmac']=vmac
    param['general']['logkappa']=np.log(kappa)

    #pyRaven's weakfield disk integration function
    model=rav.diskint2.analytical(param,False)

    #interpolating the model to be size MCMC wants
    f=np.interp(v,model['vel'],model['flux'])
    return(f)

def model2(v,kappa,vsini,vmac):
    '''
    This function creates the line profile model that will be fit to the observed profile

    Inputs:
    kappa - value of kappa that the walker is on in parameter space
    vmac - value of vmac that the walker is on in parameter space
    v - velocity array of the actual data

    Outputs:
    f - line profile model using the weak field method at the walker's position in parameter space
    '''
    param['general']['vmac']=vmac
    param['general']['vsini']=vsini
    param['general']['logkappa']=np.log(kappa)
    param['general']['Bpole']=1000

    #pyRaven's weakfield disk integration function
    model=rav.diskint2.numerical(param,False)

    #interpolating the model to be size MCMC wants
    f=np.interp(v,model['vel'],model['flux'])
    h=np.interp(v,model['vel'],model['V'])
    return(f,h)

In [ ]:
with PdfPages('{}novmacfixed.pdf'.format(path)) as pdf:
    j=-1
    for i,StarInfo in StarData.iterrows():
        if StarInfo['pyRavenFlag']==StarInfo['AsifgoodFlag']==1.0:
            j+=1
            star=StarInfo['Name']
            fname='{}05-RavenSetup/hybrid_maskdepth0.02_tellClean/{}.h5'.format(path,star)
            datapacket=datapacket=rav.data.read_packet(fname)

            fig, ax = plt.subplots(2,1, figsize=(10,10))
            for n in range(datapacket.nobs):
                x1=datapacket.scaled.lsds[n].vel#+datapacket.vrad[n]
                y1=datapacket.scaled.lsds[n].specI
                y2=datapacket.scaled.lsds[n].specV
                if n==0:
                    ax[1].plot(x1,y1,c='C0',label='Data')
                    ax[0].plot(x1,y2,c='C0')
                else:
                    ax[1].plot(x1,y1,c='C0')
                    ax[0].plot(x1,y2,c='C0')
            x2=datapacket.scaled.lsds[0].vel#+datapacket.vrad[0]
            modelout1,stokesV1=model2(x2,fitfixed[1][j],fitfixed[3][j],fitfixed[2][j])
            modelout2,stokesV2=model2(x2,fit[1][j],fit[2][j],0.0)
            ax[1].plot(x2,modelout2,c='C1',label='Fit Model Fixed vsini, no vmac')
            ax[1].plot(x2,modelout1,c='C2',label='Fit Model Fixed vsini')
            ax[0].plot(x2,stokesV2,c='C1')
            ax[0].plot(x2,stokesV1,c='C2')
            ax[1].legend()
            ax[1].set_title(star)

            pdf.savefig(fig)
        

In [ ]:
fitfixed

In [ ]:

star='hd11415'
fname='{}05-RavenSetup/hybrid_maskdepth0.02_tellClean/{}.h5'.format(path,star)
datapacket=datapacket=rav.data.read_packet(fname)

fig, ax = plt.subplots(1,1, figsize=(5,5))
for n in range(datapacket.nobs):
    x1=datapacket.scaled.lsds[n].vel#+datapacket.vrad[n]
    y1=datapacket.scaled.lsds[n].specI
    if n==0:
        ax.plot(x1,y1,c='C0',label='Data')
    else:
        ax.plot(x1,y1,c='C0')
x2=datapacket.scaled.lsds[0].vel#+datapacket.vrad[0]
modelout1=model2(x2,fitfixed[1][0],fitfixed[3][0],fitfixed[2][0])
modelout2=model2(x2,fit[1][0],fit[2][0],fit[3][0])
ax.plot(x2,modelout2,c='C1',label='Fit Model')
ax.plot(x2,modelout1,c='C2',label='Fit Model Fixed vsini')
ax.legend()
ax.set_title(star)

        

In [ ]:
param

In [ ]:
def model2(v,kappa,vsini,vmac):
    '''
    This function creates the line profile model that will be fit to the observed profile

    Inputs:
    kappa - value of kappa that the walker is on in parameter space
    vmac - value of vmac that the walker is on in parameter space
    v - velocity array of the actual data

    Outputs:
    f - line profile model using the weak field method at the walker's position in parameter space
    '''
    param['general']['vmac']=vmac
    param['general']['vsini']=vsini
    param['general']['logkappa']=np.log(kappa)

    #pyRaven's weakfield disk integration function
    model=rav.diskint2.analytical(param,False)

    #interpolating the model to be size MCMC wants
    f=np.interp(v,model['vel'],model['flux'])
    return(f)

In [ ]:

star='hd56139'
fname='{}05-RavenSetup/normalized_maskdepth0.02_tellClean/{}.h5'.format(path,star)
datapacket=datapacket=rav.data.read_packet(fname)
guess=np.array([1.3,datapacket.vsini,30])

parameters,covariance,modelout = rav.fitparams.fitdata(param,datapacket,guess)

fig, ax = plt.subplots(1,1, figsize=(5,5))
for n in range(datapacket.nobs):
    x1=datapacket.scaled.lsds[n].vel#+datapacket.vrad[n]
    y1=datapacket.scaled.lsds[n].specI
    if n==0:
        ax.plot(x1,y1,c='C0',label='Data')
    else:
        ax.plot(x1,y1,c='C0')
x2=datapacket.scaled.lsds[0].vel#+datapacket.vrad[0]
ax.plot(x2,modelout,c='C1',label='Fit Model')
ax.legend()
ax.set_title(star)


In [ ]:
parameters